In [2]:
from transformers import BertTokenizer, BertModel

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [22]:
text = '2\nH. GIMBERT AND F. HORN\na\nb\nc\nd\n.4\n.6\n.3\n.1\n.4\n.1\n.1\n.2\n.8\n.2\n.6\n.2\nFigure 1: A Simple Stochastic Game.\ndepends on who can influence the immediate evolution of the system: it is a Min vertex if the\nsoftware can choose between different options, a Max vertex if there is a (non-deterministic)\ninput asked from the user, and a random vertex if the evolution depends on a stochastic\nenvironment. An optimal strategy for Min can then be used as the basis for the synthesis\nof a “good” driver, i.e. one which minimises the probability of entering the error state\nindependently of the behaviour of the user.\nThe main algorithmic problem about SSGs is the computation of values of the vertices\nand optimal strategies for the players. This problem was first adressed by Condon, who\nshowed that deciding whether the value of a vertex is greater than 1\n2 belongs to NP and\nco-NP [Con92].\nCondon’s algorithm guesses non-deterministically the values of vertices,\nwhich are rational numbers of linear size, and checks that they are solutions of some local\noptimality equations. This algorithm is correct only for stopping games, where the pebble\nreaches either the target or a sink target with probability one, regardless of the players’\nstrategies.\nAny SSG can be transformed in polynomial time into a stopping SSG with\n(almost) the same values, but it incurs a quadratic blow-up of the size of the game.\nThree other algorithms for solving SSGs are presented in [Con93]. The first one com-\nputes the values of the vertices using a quadratic program with linear constraints. The\nsecond one computes iteratively from below the values of the vertices, and the third is a\nstrategy improvement algorithm `\na la Hoffman-Karp [HK66]. The two latter algorithms,\nas the ones recently proposed in [Som05], solve a series of linear programs which could be\nof exponential length. Furthermore, solving a linear program requires high-precision arith-\nmetic, even if it can be done in polynomial time [Kha79, Ren88]. The best randomised\nalgorithms achieve sub-exponential expected time eO(√n) [Lud95, Hal07].\nIn this paper we present two algorithms computing the values and optimal strategies\nin SSGs: the “permutation-enumeration” and the “permutation-improvement” algorithms.\nThe common basis for both algorithms is that optimal strategies can be looked for in a\nsubset of the positional strategies called permutation strategies. Permutation strategies are\nderived from permutations over the random vertices. In order to find optimal strategies, the\npermutation-enumeration algorithm performs an exhaustive search among all permutation\nstrategies, whereas the permutation-improvement algorithm performs successive improve-\nments of permutation strategies, `\na la Hoffman-Karp [HK66].'
len(text.split())

438

In [35]:
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input.attention_mask.shape)


torch.Size([1, 637])


In [24]:
output = model(**encoded_input)
#print(output)
print(output.pooler_output.shape)

RuntimeError: The size of tensor a (637) must match the size of tensor b (512) at non-singleton dimension 1

In [30]:
import json
import torch

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
def tokenize_entry(line):
    entry = json.loads(line)
    abstract = entry["abstract"]
    title = entry["title"]
    joined = title + ": " + abstract
    tokenized = tokenizer(joined, return_tensors="pt")

In [36]:
def generate_chunks(encoded):
    chunk_size = 512  # BERT's maximum input size

    input_ids = encoded_input['input_ids'].squeeze(0)  # Remove the batch dimension
    attention_mask = encoded_input['attention_mask'].squeeze(0)

    input_ids_chunks = []
    attention_mask_chunks = []

    for i in range(0, len(input_ids), chunk_size - 2):  # Account for special tokens
        # Extract segments of the tokenized input
        input_ids_chunk = input_ids[i:i + chunk_size - 2]
        attention_mask_chunk = attention_mask[i:i + chunk_size - 2]

        # Add special tokens ([CLS] and [SEP]) to the chunks
        input_ids_chunk = torch.cat([
            torch.tensor([tokenizer.cls_token_id]),  # [CLS] at the beginning
            input_ids_chunk,
            torch.tensor([tokenizer.sep_token_id])  # [SEP] at the end
        ])
        attention_mask_chunk = torch.cat([
            torch.tensor([1]),  # Attention mask for [CLS]
            attention_mask_chunk,
            torch.tensor([1])  # Attention mask for [SEP]
        ])

        # Pad the chunks to the chunk size
        padding_length = chunk_size - input_ids_chunk.size(0)
        if padding_length > 0:
            input_ids_chunk = torch.cat([input_ids_chunk, torch.zeros(padding_length, dtype=torch.long)])
            attention_mask_chunk = torch.cat([attention_mask_chunk, torch.zeros(padding_length, dtype=torch.long)])

        input_ids_chunks.append(input_ids_chunk.unsqueeze(0))  # Add batch dimension
        attention_mask_chunks.append(attention_mask_chunk.unsqueeze(0))  # Add batch dimension

    input_ids_chunks = torch.cat(input_ids_chunks, dim=0)
    attention_mask_chunks = torch.cat(attention_mask_chunks, dim=0)

    return input_ids_chunk, attention_mask_chunk

In [ ]:
def generate_embedding(input_chunks, attention_chunks):
    for input_chunk, attention_chunk in zip(input_chunk, attention_chunk):
        input = {
            "input_ids": input_chunk,
            "attention_mask": attention_chunk
        }
        model(**input)

    # Retrun aggreate

In [ ]:
def index_embedding(embedding, title, id, categories):
    # Add the embedding to the indexing system
    
    pass

In [ ]:
with open("./ml-papers.txt", "r") as f:
    for line in f:
        tokenized = tokenize_entry(line)
        inputs, attentions = generate_chunks(tokenized)
        embedding = generate_embedding(inputs, attentions)
        